In [1]:
import pandas as pd
import numpy as np
import os

from utils.preprocessing import preprocess_df
from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df, load_breast_cancer_df
from utils.evaluation import get_evaluations, EvaluationMatrix

# from utils.load import load_result_from_csv, load_datapoints_from_npy
# from utils.metric import metric_generator

seed = 123

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None # suppress "SettingWithCopyWarning" warning

In [3]:
datasets = [
        "adult",
        "german",
        "compas",
        "diabetes",
        "breast_cancer",
        ]

models = ["dt","rfc","lr","svc","nn","nn_2"]


attack_list = [
        'deepfool', 
        'carlini_l_2', 'carlini_l_inf', 
        'lowprofool_l_2', 'lowprofool_l_inf', 
        'boundary', 
        'hopskipjump_l_2', 'hopskipjump_l_inf'
        ]


In [4]:
def get_loading_fn(dataset_name):
    if dataset_name == 'adult':
        dataset_loading_fn = load_adult_df
    elif dataset_name == 'german':
        dataset_loading_fn = load_german_df
    elif dataset_name == 'compas':
        dataset_loading_fn = load_compas_df
    elif dataset_name == 'diabetes':
        dataset_loading_fn = load_diabetes_df
    elif dataset_name == 'breast_cancer':
        dataset_loading_fn = load_breast_cancer_df
    else:
        raise Exception("Unsupported dataset")
    return dataset_loading_fn


In [22]:
#### Select dataset ####

all_metric = {}

for dataset_name in datasets:
    all_metric[dataset_name]={}

    df_info = preprocess_df(get_loading_fn(dataset_name))
    for attack in attack_list:
        all_metric[dataset_name][attack]={}

        folder_name = f'{attack}_{dataset_name}'
        for model_name in models:

            file_name = f'{folder_name}_{model_name}_result.csv'
            result_path = f'./results/{folder_name}/{file_name}'
            if os.path.isfile(result_path):
                result_df = pd.read_csv(result_path)
                evaluation_df, metric = get_evaluations(result_df, 
                    df_info, 
                    matrix = [
                        EvaluationMatrix.L1, 
                        EvaluationMatrix.L2, 
                        EvaluationMatrix.Linf,
                        EvaluationMatrix.Sparsity, 
                        # EvaluationMatrix.Realistic, 
                        EvaluationMatrix.MAD, 
                        EvaluationMatrix.Mahalanobis,
                        # EvaluationMatrix.Perturbation_Sensitivity,
                        EvaluationMatrix.Neighbour_Distance,
                        ])
                
                all_metric[dataset_name][attack][model_name] = metric

                csv_save_result_path = f'results/{folder_name}/eval_{file_name}'
                evaluation_df.to_csv(csv_save_result_path)
                print(f"Have saved file to {csv_save_result_path}")


Have saved file to results/deepfool_adult/eval_deepfool_adult_lr_result.csv
Have saved file to results/deepfool_adult/eval_deepfool_adult_svc_result.csv
Have saved file to results/deepfool_adult/eval_deepfool_adult_nn_2_result.csv
Have saved file to results/carlini_l_2_adult/eval_carlini_l_2_adult_lr_result.csv
Have saved file to results/carlini_l_2_adult/eval_carlini_l_2_adult_svc_result.csv
Have saved file to results/carlini_l_2_adult/eval_carlini_l_2_adult_nn_2_result.csv
Have saved file to results/carlini_l_inf_adult/eval_carlini_l_inf_adult_lr_result.csv
Have saved file to results/carlini_l_inf_adult/eval_carlini_l_inf_adult_svc_result.csv
Have saved file to results/carlini_l_inf_adult/eval_carlini_l_inf_adult_nn_2_result.csv
Have saved file to results/boundary_adult/eval_boundary_adult_dt_result.csv
Have saved file to results/boundary_adult/eval_boundary_adult_rfc_result.csv
Have saved file to results/boundary_adult/eval_boundary_adult_lr_result.csv
Have saved file to results/bou

In [34]:
df_metric = pd.DataFrame(columns=["dataset",  "attack", "model", "metric", "value"])

for dataset, dic1 in all_metric.items():
    for attack, dic2 in dic1.items():
        for model, dic3 in dic2.items():
            for metric, value in dic3.items():
                df_metric.append({"dataset":dataset, "attack":attack, "model":model, "metric":metric, "value":value}, ignore_index=True)



In [33]:
df_metric

,dataset,model,attack,metric,value


In [21]:
all_metric

{'adult': {'deepfool': {'lr': {'L1': 0.8174119,
    'L2': 0.444698,
    'Linf': 0.36128974,
    'Sparsity': 8.28125,
    'MAD': 5.597751,
    'Mahalanobis': 0.05807913,
    'Neighbour_Distance': 0.38671675},
   'svc': {'L1': 0.1485034,
    'L2': 0.11723521,
    'Linf': 0.11307675,
    'Sparsity': 8.34375,
    'MAD': 1.4672297,
    'Mahalanobis': 0.0098925335,
    'Neighbour_Distance': 0.09155505},
   'nn_2': {'L1': 1.6526703,
    'L2': 0.5417245,
    'Linf': 0.27857846,
    'Sparsity': 7.84375,
    'MAD': 4.6453214,
    'Mahalanobis': 0.13605897,
    'Neighbour_Distance': 0.44511637}},
  'carlini_l_2': {'lr': {'L1': 0.7840552,
    'L2': 0.509782,
    'Linf': 0.39373115,
    'Sparsity': 8.34375,
    'MAD': 3.7742414,
    'Mahalanobis': 0.07280562,
    'Neighbour_Distance': 0.40951678},
   'svc': {'L1': 0.00042951127,
    'L2': 0.0002561772,
    'Linf': 0.0001886134,
    'Sparsity': 9.859375,
    'MAD': 0.002579279,
    'Mahalanobis': 3.374673e-05,
    'Neighbour_Distance': 0.0002561772}

In [20]:
df = pd.json_normalize(all_metric, )
df


,adult.deepfool.lr.L1,adult.deepfool.lr.L2,adult.deepfool.lr.Linf,adult.deepfool.lr.Sparsity,adult.deepfool.lr.MAD,adult.deepfool.lr.Mahalanobis,adult.deepfool.lr.Neighbour_Distance,adult.deepfool.svc.L1,adult.deepfool.svc.L2,adult.deepfool.svc.Linf,...,breast_cancer.hopskipjump_l_inf.nn.MAD,breast_cancer.hopskipjump_l_inf.nn.Mahalanobis,breast_cancer.hopskipjump_l_inf.nn.Neighbour_Distance,breast_cancer.hopskipjump_l_inf.nn_2.L1,breast_cancer.hopskipjump_l_inf.nn_2.L2,breast_cancer.hopskipjump_l_inf.nn_2.Linf,breast_cancer.hopskipjump_l_inf.nn_2.Sparsity,breast_cancer.hopskipjump_l_inf.nn_2.MAD,breast_cancer.hopskipjump_l_inf.nn_2.Mahalanobis,breast_cancer.hopskipjump_l_inf.nn_2.Neighbour_Distance
0,0.817412,0.444698,0.36129,8.28125,5.597751,0.058079,0.386717,0.148503,0.117235,0.113077,...,5.224981e-08,6.831826e-09,4.459362e-08,1.889484,0.358745,0.07202,0.0625,0.650798,0.101937,0.335349


In [7]:
# #### Select dataset ####
# for dataset_name in datasets: 
#     All_Results[dataset_name] = {}
#     All_Datapoints[dataset_name] = {}

#     for model in models:
#         All_Results[dataset_name][model] = {}
#         All_Datapoints[dataset_name][model] = {}

#         for attack in attack_list:

#             All_Results[dataset_name][model][attack] = load_result_from_csv(attack, dataset_name, model)

#             All_Datapoints[dataset_name][model][attack] = {
#                 "original": load_datapoints_from_npy(attack, dataset_name, model, False),
#                 "adv": load_datapoints_from_npy(attack, dataset_name, model, True),
#             }


In [8]:
# All_Metrics = {}

# for dataset_name in datasets: 
#     All_Metrics[dataset_name] = {}
#     for model in models: 
#         All_Metrics[dataset_name][model] = {}
#         for attack in attack_list:

#             if All_Results[dataset_name][model][attack] is not None:
#                 if All_Results[dataset_name][model][attack]["original"] is not None:
#                     if All_Results[dataset_name][model][attack]["adv"] is not None:
#                         All_Metrics[dataset_name][model][attack] = metric_generator(
#                             All_Results[dataset_name][model][attack],
#                             All_Results[dataset_name][model][attack]["original"],
#                             All_Results[dataset_name][model][attack]["adv"]
#                         )
